# src - optimizer

## Notebook运行提示
- 代码已拆分为多个小单元, 按顺序运行即可在每一步观察输出与中间变量。
- 涉及 `Path(__file__)` 或相对路径的脚本会自动注入 `__file__` 解析逻辑, Notebook 环境下也能引用原项目资源。
- 可在每个单元下追加说明或参数试验记录, 以跟踪核心算法和数据处理步骤。


In [ ]:
from .parameters import get_grouped_llrd_parameters, get_optimizer_params
from torch.optim import AdamW
from torchcontrib.optim import SWA

In [ ]:


def get_optimizer(model, config):
    
    if config.optimizer.group_lt_multiplier == 1:
        optimizer_parameters = get_optimizer_params(model,
                                                    config.optimizer.encoder_lr,
                                                    config.optimizer.decoder_lr,
                                                    weight_decay=config.optimizer.weight_decay)
    else:
        optimizer_parameters = get_grouped_llrd_parameters(model,
                                                           encoder_lr=config.optimizer.encoder_lr,
                                                           decoder_lr=config.optimizer.decoder_lr,
                                                           embeddings_lr=config.optimizer.embeddings_lr,
                                                           lr_mult_factor=config.optimizer.group_lt_multiplier,
                                                           weight_decay=config.optimizer.weight_decay,
                                                           n_groups=config.optimizer.n_groups)

    optimizer = AdamW(optimizer_parameters,
                      lr=config.optimizer.encoder_lr,
                      eps=config.optimizer.eps,
                      betas=config.optimizer.betas)

    if config.optimizer.use_swa:
        optimizer = SWA(optimizer,
                        swa_start=config.optimizer.swa.swa_start,
                        swa_freq=config.optimizer.swa.swa_freq,
                        swa_lr=config.optimizer.swa.swa_lr)
    return optimizer